In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision  
import subprocess
import cv2
import ffmpeg
import os
import os.path as osp
import torchvision.models as models

from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [49]:
VID_PATH = "../../data/extracted_videos/"
VID_PATH_OG = "../../data/videos_new/"
LABEL_FILE = "../../data/data.npy"
PROCESSED_PATH = "../../data_temp/processed/"
DATA_SAVE_PATH = "../../data_temp/labeled_videos/"
MODELS_PATHS = "./models/"
# LABEL_PATH = '../../data/labels/'
FRAME_RATE = 2

In [47]:
"""
  Preprocess video data.
"""
def label_map(lab):
    if(lab == 0):
        return 2
    elif(lab == -1):
        return 0
    else:
        return 1
    
def get_all_files_from_dir(directory, vids = False):
    file_paths = []
    print(directory)
    try:
        for root, dirs, files in os.walk(directory):
            # print(files)
            if(vids):
                file_paths += [os.path.join(root, x,x+".mp4") for x in dirs]
            else:
                file_paths += [os.path.join(root, x) for x in files]
        return sorted(file_paths)
    except Exception as e:
        print(e)
    
def process_video(video_file, labels):
    video_filename = ".".join(video_file.split('/')[-1].split('.')[:-1])
    vidcap = cv2.VideoCapture(video_file)

    ctr = 0
    video_frames = []
    video_labels = []
    
    hasFrames,image = vidcap.read()

    while (hasFrames):
        save_file_name = video_filename + "_" + str(ctr) + ".npy"
        np.save(osp.join(PROCESSED_PATH, save_file_name), image)  
        label_ts = vidcap.get(cv2.CAP_PROP_POS_MSEC) + 1000 #take 1 sec ahead labels 
        label_ts = label_ts - (label_ts%100)
        if(label_ts not in labels.keys()):
            print(label_ts)
            hasFrames,image = vidcap.read()
            continue
        label = labels[label_ts]
        video_labels.append(label_map(label))
        video_frames.append(save_file_name)
        hasFrames,image = vidcap.read()
        ctr += 1
        
    df = pd.DataFrame({'frames': video_frames, 'labels': video_labels})
    df.to_csv(osp.join(DATA_SAVE_PATH,video_filename+".csv"), index=None)

    print("After processing:")
    print("Number of frames labelled: ", ctr)
    
def preprocess():
    f = np.load(LABEL_FILE, allow_pickle = True)
    for video_file in get_all_files_from_dir(VID_PATH):
        video_filename = ".".join(video_file.split('/')[-1].split('.')[:-1])
        print(video_filename)
        labels = f[video_filename]['Sensor']['direction_label']['direction']
        process_video(video_file, labels)
        print("Finished processing ", video_file)
        
def process_videos(vid_path = VID_PATH_OG):
    fp = get_all_files_from_dir(vid_path, vids=True)
    print(fp)
    for fl in fp:
        video_filename = fl.split('/')[-1]
        ffmpeg.input(fl).filter('fps', fps=10, round='up').output(VID_PATH+video_filename).run() 

In [48]:
### preprocess videos
# process_videos("../../data/videos_temp/")
preprocess()

../../data/extracted_videos_new/
2022-04-04T18_41_15.478Z


KeyError: '2022-04-04T18_41_15.478Z'

In [50]:
BATCH = 64
SEQUENCE_LENGTH = 10
HEIGHT = 128
WIDTH = 128
CHANNELS = 3

In [91]:
def save(model, index, optim = False):
    if not os.path.exists(MODELS_PATHS+'/attempt_10_frames_resnet34_new_data_diff_split'):
        os.mkdir(MODELS_PATHS+'/attempt_10_frames_resnet34_new_data_diff_split')
    if(optim):
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt_10_frames_resnet34_new_data_diff_split'+'/optimizer_params_{:08d}.pth'.format(index))
    else:
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt_10_frames_resnet34_new_data_diff_split'+'/model_params_{:08d}.pth'.format(index))

In [53]:
class ResNet18(nn.Module):
    """
    Container for ResNet50 s.t. it can be used for metric learning.
    The Network has been broken down to allow for higher modularity, if one wishes
    to target specific layers/blocks directly.
    """

    def __init__(self, fixconvs=False, pretrained=True):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=pretrained)
        if fixconvs:
            for param in self.model.parameters():
                param.requires_grad = False

        self.regressor = nn.Linear(self.model.fc.in_features, 3)
        self.dropout = torch.nn.Dropout(p=0.05)
        self.model = torch.nn.Sequential(*(list(self.model.children())[:-1]))
        # model.fc.weight.requires_grad = True
        # model.fc.bias.requires_grad = True

    def forward(self, x):
        x = self.model(x)
        x = torch.squeeze(x)
        x = self.dropout(x)
        x = self.regressor(x)
        return x

In [54]:
class FrameDataset(Dataset):
    def __init__(self, x, y, transforms, base_path):
        self.transforms = transforms
        self.X = x
        self.y = y
        # self.seq_len = seq_len
        self.base_path = base_path
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        seq_filename = self.X[idx]
        try:
            frame = np.load(osp.join(self.base_path,seq_filename), allow_pickle=True)
            frame = (frame - frame.min())/(frame.max() - frame.min())
            frame = self.transforms(frame)
            
        except Exception as ex:
            print("Error occured while loading frame: ", ex)
            frame = torch.zeros((CHANNELS, HEIGHT, WIDTH))
        
        return frame, self.y[idx]
        

In [83]:
def make_tt_split(data_folder):
    X = []
    y = []
    
    for filename in os.listdir(data_folder):
        if(filename[-3:]=="csv"):
            df = pd.read_csv(osp.join(data_folder,filename))
            X.append(df['frames'].to_numpy())
            y.append(df['labels'].to_numpy())
    
   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    print(len(X_train))
    print(len(X_test))
    
    X_train = np.concatenate(X_train)
    # print(X.head())
    # X_train.reset_index(drop=True,inplace=True)
    # X_train = X_train.to_numpy()

    
    y_train = np.concatenate(y_train)
    # y_train.reset_index(drop=True,inplace=True)
    # y_train = y_train.to_numpy()
    
    
    X_test = np.concatenate(X_test)
    # print(X.head())
    # X_test.reset_index(drop=True,inplace=True)
    # X_test = X_test.to_numpy()

    
    y_test = np.concatenate(y_test)
    # y_test.reset_index(drop=True,inplace=True)
    # y_test = y_test.to_numpy()
    
    print(len(X_test))
    print(len(y_test))
    print(X_test)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test


In [84]:
cuda = torch.cuda.is_available()
print(cuda)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# train_transforms = [ttf.ToTensor(), transforms.Resize((HEIGHT, WIDTH)), transforms.ColorJitter(), transforms.RandomRotation(10), transforms.GaussianBlur(3)]
train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])
val_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])

X_train, X_test, y_train, y_test = make_tt_split(DATA_SAVE_PATH)
train_dataset = FrameDataset(X_train, y_train, transforms=train_transforms, base_path = PROCESSED_PATH)
val_dataset = FrameDataset(X_test, y_test, transforms=val_transforms, base_path = PROCESSED_PATH)

train_args = dict(shuffle=True, batch_size=BATCH, num_workers=1, pin_memory=True, drop_last=False) if cuda else dict(shuffle=True, batch_size=BATCH, drop_last=False)
train_loader = DataLoader(train_dataset, **train_args)

val_args = dict(shuffle=False, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=BATCH, drop_last=False)
val_loader = DataLoader(val_dataset, **val_args)



True
12
4
1197
1197
['2022-04-04T16:12:56_0.npy' '2022-04-04T16:12:56_1.npy'
 '2022-04-04T16:12:56_2.npy' ... '2022-04-04T16:20:59_535.npy'
 '2022-04-04T16:20:59_536.npy' '2022-04-04T16:20:59_537.npy']


In [85]:
print(len(train_dataset))
print(len(val_dataset))

3122
1197


In [86]:
def validate(val_loader, val_dataset, model):
    # validation
    model.eval()
    val_num_correct = 0
   
    for i, (vx, vy) in enumerate(val_loader):
      
        vx = vx.float().to(device)
        vy = vy.to(device)

        with torch.no_grad():
            outputs = model(vx)
            del vx

        val_num_correct += int((torch.argmax(outputs, axis=1) == vy).sum())
        del outputs
        # break
    

    print("Validation: {:.04f}%".format(100 * val_num_correct / (len(val_dataset))))


In [92]:
lr = 0.008 #changed from 0.01
epochs = 25
lamda = 1e-2  #L2 regularization #changed from 1e-4
num_classes = 3
convlstm_hidden = 128
num_conv_lstm_layers = 2

model = ResNet18()
# model.load_state_dict(torch.load('./models/attempt_7_frames_resnet34_new_data/model_params_00000003.pth'))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=lamda, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./models/attempt_7_frames_resnet34_new_data/optimizer_params_00000003.pth'))

# for g in optimizer.param_groups:
#     g['lr'] = lr
    # g['weight_decay']= lamda
    
scaler = torch.cuda.amp.GradScaler()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# print(model)

In [94]:
for epoch in range(epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0
    
    for i, (x, y) in enumerate(train_loader):
       
        model.train()
        optimizer.zero_grad()

        x = x.float().to(device)
        y = y.to(device)
        
        with torch.cuda.amp.autocast():
            outputs = model(x)
            del x
            loss = criterion(outputs.view(-1,num_classes), y.long().view(-1))

        # print(outputs.shape)
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        del outputs
        total_loss += float(loss)

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * BATCH)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        scaler.scale(loss).backward()
        scaler.step(optimizer) 
        scaler.update()

        scheduler.step()

        batch_bar.update() # Update tqdm bar
        # break
        

    batch_bar.close()

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / (len(train_dataset)),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    
    save(model, epoch)
    save(optimizer, epoch, optim=True)
    
    validate(val_loader, val_dataset, model)
    
batch_bar.close()

KeyboardInterrupt: 

In [38]:
def prep_video_test(filename):
    X = []
    y = []
    
    df = pd.read_csv(filename)
    X.append(df['frames'])
    y.append(df['labels'])   
    
    X = pd.concat(X)    # print(X.head())
    X.reset_index(drop=True,inplace=True)
    X = X.to_numpy()

    
    y = pd.concat(y)
    y.reset_index(drop=True,inplace=True)
    y = y.to_numpy()
    
    return X, y, df

def validate_test(val_loader, val_dataset, model):
    # validation
    model.eval()
    val_num_correct = 0
    predictions = []
    
    for i, (vx, vy) in enumerate(val_loader):
      
        vx = vx.float().to(device)
        vy = vy.to(device)

        with torch.no_grad():
            outputs = model(vx)
            del vx

        preds = torch.argmax(outputs, axis=1)
        predictions.append(preds.cpu().detach().numpy())
        val_num_correct += int((preds == vy).sum())
        del outputs
    
        # val_num_correct += int((torch.argmax(outputs, axis=1) == vy).sum())
        # del outputs
        # break
    
    # print(predictions)
    predictions = np.concatenate(predictions)
    acc = 100 * val_num_correct / (len(val_dataset))
    print("Validation: {:.04f}%".format(acc))
    return predictions, acc
    

In [39]:
X, y, df = prep_video_test('../../data_temp/labeled_videos/2022-04-04T18_41_15.csv')
test_dataset = FrameDataset(X, y, transforms=val_transforms, base_path = PROCESSED_PATH)
val_args = dict(shuffle=False, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=BATCH, drop_last=False)
test_loader = DataLoader(test_dataset, **val_args)
print(len(test_dataset))
predictions, acc = validate_test(test_loader, test_dataset, model)
df['predictions'] = predictions
print(df.head())
df.to_csv("predictions_2022-04-04T16:07:08_{}.csv".format(acc), index=None)


264
Validation: 99.6212%
                      frames  labels  predictions
0  2022-04-04T16:07:08_0.npy       2            2
1  2022-04-04T16:07:08_1.npy       2            2
2  2022-04-04T16:07:08_2.npy       2            2
3  2022-04-04T16:07:08_3.npy       2            2
4  2022-04-04T16:07:08_4.npy       2            2
